# Problem: Grover search

Suppose you are presented with an unstructured Boolean formula $f:\{0,1\}^n\rightarrow\{0,1\}$ and asked whether $f(x)=1$ for any value of $x$ or $f(x)=0$ for all $x$. Without a quantum computer, you would potentially need to try all $N=2^n$ inputs ($O(N)$) to find a solution or show one does not exist. 

However, a quantum computer can use an "oracle" for $f$, i.e.~an $(n+A+1)$-qubit unitary $U_f$ such that $U_f|x,b,0^A\rangle\rightarrow|x,b\oplus f(x),0^A\rangle$, just $O(\sqrt{N})$ times to answer this question. Here $A$ is the number of scratchpad (or "ancillary") qubits that may help in implementing $U_f$ and other parts of the search algorithm.

Grover search begins with the $n$-qubit initial state
<h3>
$$|S\rangle=\frac{1}{\sqrt{N}}\sum_{x\in\{0,1\}^n}|x\rangle=\sqrt{1-\frac{m}{N}}\underbrace{\left(\frac{1}{\sqrt{N-m}}\sum_{x:f(x)=0}|x\rangle\right)}_{=|\overline{T}\rangle}+\sqrt{\frac{m}{N}}\underbrace{\left(\frac{1}{\sqrt m}\sum_{x:f(x)=1}|x\rangle\right)}_{=|T\rangle},$$
</h3>
where $m$ is the number of inputs $x$ on which $f$ evaluates to $1$, and we call $|T\rangle$ the "target state" and $|\overline{T}\rangle$ the "non-target state". Interestingly, despite being an algorithm operating on $n+A+1$ qubits, Grover search takes place only in a small subspace of the full Hilbert space -- the subspace $\mathcal{T}$ spanned by $|\overline{T}\rangle\otimes|0^{A+1}\rangle$ and $|T\rangle\otimes|0^{A+1}\rangle$. Let us write $|S\rangle\otimes|0^{A+1}\rangle=\left(\sqrt{1-m/N},\sqrt{m/N}\right)$ as a 2-dimensional vector in this space.

Here are two unitaries (called "reflections") acting on the subspace $\mathcal{T}$.
<h3>
$$ R_S(\alpha) = Id - (1-e^{-i\alpha})|S\rangle\langle S|=\begin{bmatrix}1-(1-e^{-i\alpha})(1-\frac{m}{N}) & -(1-e^{-i\alpha})\sqrt{1-\frac{m}{N}}\sqrt{\frac{m}{N}} \\ -(1-e^{-i\alpha})\sqrt{1-\frac{m}{N}}\sqrt{\frac{m}{N}} & 1-(1-e^{-i\alpha})\frac{m}{N}\end{bmatrix},\\
R_T(\beta) = Id - (1-e^{i\beta})|T\rangle\langle T| = \begin{bmatrix}1 & 0 \\ 0 & e^{i\beta}\end{bmatrix}.$$
</h3>
The matrix elements here are the values of $\langle T|R_S(\alpha)|T\rangle$, $\langle T|R_S(\alpha)|\overline{T}\rangle$, $\langle\overline{T}| R_S(\alpha)|T\rangle$, and $\langle\overline{T}|R_S(\alpha)|\overline{T}\rangle$, and likewise for $R_T(\beta)$. Implementations of $R_S$ and $R_T$ are any $(n+A+1)$-qubit circuits that act as shown by these $2\times 2$ matrices on the subspace $\mathcal{T}$.

For instance, one might implement $R_T(\beta)$ by creating a circuit that imparts a phase factor $e^{i\beta}$ to the states (and only the states) $|x\rangle$ for which $f(x)=1$. One way (though not the most efficient way) this can be done is by using $U_f$ to compute the value of $f$ into an ancilla, applying an appropriate single-qubit unitary to the ancilla, and then using $U_f$ a second time to uncompute the value of $f$ from the ancilla. The implementation of $R_S(\alpha)$ can also be done analogously as one can tell from its similarity with $R_T$, i.e.~$R_S$ imparts a phase $e^{-i\alpha}$ to only the $|S\rangle$ state.

Recall that the product of two unitaries $U_2U_1$ indicates that $U_1$ is applied first, then $U_2$. One way to build a circuit for $U_2U_1$ is to make circuits for $U_1$ and $U_2$ individually and then apply them sequentially.

We consider just an $n=3$ example, for a total of $4+A$ qubits. Your task is to implement the "Grover iterate", the $(4+A)$-qubit unitary $R_S(\alpha)R_T(\beta)$, for arbitrary real parameters $\alpha$ and $\beta$ and several choices of the function $f$. You must do so for five different Boolean functions $f_0,f_1,f_2,f_3$, and $f_4$ where $f_m$ has $m$ solutions to $f(x)=1$, $x=x_1x_2x_3\in\{0,1\}^3$. Evaluated modulo 2, we choose
\begin{align}\label{eq:fs}
f_0(x_1x_2x_3)&=0,\quad f_1(x_1x_2x_3)=x_1x_2x_3,\quad f_2(x_1x_2x_3)=x_1x_2x_3+(1-x_1)(1-x_2)(1-x_3),\\\nonumber
f_3(x_1x_2x_3)&=x_1+x_2+x_3+x_1x_2x_3,\quad f_4(x_1x_2x_3)=x_1+x_2+x_3.
\end{align}
For instance, $f_2$ returns 1 only when all three bits match, so it has two solutions, $x=000$ and $x=111$, to $f_2(x)=1$. Note that all functions treat the individual bits of the input $x_1,x_2,x_3$ symmetrically, or in other words, permuting the inputs does not affect the output. You may find it easiest to implement the oracle $U_{f_m}$ (as described above) first, then use it as a sub-circuit to implement the Grover iterate, but this is not strictly necessary.


<h1>
    $f_m$ for m = 0

In [4]:
# Importing the qiskit module
from qiskit import *

# Defining input, output and scratch qubits
x1 = 3    # number of input qubits
y1 = 1   # number of output qubit 
s1 =  1   # number of scratch qubit
m = 0  # for m = 0
fm = []
# Defining Quantum Circuit with the given circuits
def Circuit_0(In,Ou,Sc):
    if Sc != 0:
        # initiating required qubits
        X = QuantumRegister(In, 'input') 
        Y1= QuantumRegister(Ou, 'output') 
        S = QuantumRegister(Sc, 'scratch')  
        
        # creating circuit with above qubits
        Circ = QuantumCircuit(X,Y1,S)
    else:
        
        # initiating required qubits
        X = QuantumRegister(In, 'input') 
        Y1= QuantumRegister(Ou, 'output') 
        
        # creating circuit with above qubits
        Circ = QuantumCircuit(X,Y1)
    
    ##### Create you circuit below #########
    
    Circ.h(X)
    Circ.h(X)
    Circ.x(X)
    Circ.mcp(beta,X[0,1],Y1)
    Circ.x(X)
    Circ.h(X)

    
    ########################################
    
    # Uncomment to draw quantum circuit
    display(Circ.draw('mpl'))
    
    # Transpiling the circuit into u, cnot
    Circ = transpile(Circ, basis_gates=['u3','cx'])
    
    # Uncomment to draw transpiled circuit
#     display(Circ.draw('mpl'))
    
    return Circ

<h1>
    $f_m$ for m = 1

<h1>
    $f_m$ for m = 2

<h1>
    $f_m$ for m = 3

<h1>
    $f_m$ for m = 4

In [3]:
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np
from qiskit import Aer, execute
class SubmissionError(BaseException):
    def __init__(self, message):
        self.message = message
    def __str__(self):
        return self.message
def produce_unitary(m, alpha, beta):
    """Creates a unitary matrix for comparison with submissions.
        Args:
            m (int): Index of boolean function to create oracle for (see question).
            alpha (float): Angle as per question.
            beta (float): Angle as per question.
            naux (int): Number of auxiliary (scratch) qubits.
        Returns:
            ndarray: 2**(m+naux) * 2**(m+naux) unitary matrix, with phase
                     of first element == 0.
    """
    iden = np.identity(8, dtype=complex)
    targets = {0: [],  # no solutions
               1: ['111'],
               2: ['111', '000'],
               3: ['100','010','001'],
               4: ['100','010','001','111']
              }
    # Create Oracle Unitary
    oracle_matrix = iden
    for state in targets[m]:
        idx = int(state,2)
        oracle_matrix[idx][idx] = np.exp(alpha*1j)
    # Create Diffuser Unitary
    diff_qc = QuantumCircuit(3)
    diff_qc.h([0,1,2])
    diff_qc.x([0,1,2])
    diff_qc.mcp(beta,[0,1], 2)
    diff_qc.x([0,1,2])
    diff_qc.h([0,1,2])
    usim = Aer.get_backend('unitary_simulator')
    diff_matrix = execute(diff_qc, usim).result().get_unitary()
    # Combine into iterator
    iterate_matrix =  diff_matrix @ oracle_matrix
    # Set phase of first element to 0 for comparison with submission
    iterate_matrix = iterate_matrix*np.exp(-np.angle(iterate_matrix[0][0])*1j)
    return iterate_matrix
def is_correct(qc, m):
    """Args:
           qc (QuantumCircuit): QuantumCircuit to check validity of. Input register must
                                have name 'input' and size 3, all other qubits will be
                                treated as scratch qubits. Circuit must have exactly two 
                                parameters named 'alpha' and 'beta'. 
           m (int): Index of boolean function to create oracle for (see question).
       Returns:
           Bool: True if circuit is correct.        
    """
    usim = Aer.get_backend('unitary_simulator')
    svsim = Aer.get_backend('statevector_simulator')
    alpha = next(x for x in qc.parameters if x.name == 'alpha')
    beta  = next(x for x in qc.parameters if x.name == 'beta')
    nqubits = len(qc.qubits)
    if nqubits > 10:
        raise SubmissionError("Circuit cannot contain more than ten qubits.")
    # Make sure 'input' is always first
    qr = QuantumRegister(3,'input')
    pre_qc = QuantumCircuit(qr)
    qc = pre_qc + qc
    if len(qc.qubits) != nqubits:  # Probably register name mismatch
        raise SubmissionError("You must name your three-qubit input register 'input'.")
    for trials in range(30):
        param = {alpha: np.random.rand()*np.pi*2,
                  beta: np.random.rand()*np.pi*2}
        submission = execute(qc, usim, parameter_binds=[param]).result().get_unitary()
        submission = submission*np.exp(-np.angle(submission[0][0])*1j)
        submission = submission[0:8,0:8]  # matrix if all aux qubits = |0>
        is_close = np.isclose(submission,
                         produce_unitary(m, param[alpha], param[beta]))
        if False in is_close:
            raise SubmissionError(f"Circuit is incorrect (alpha = {param[alpha]}, beta = {param[beta]}).")
    return True

In [5]:
is_correct(Circuit_0(x1,y1,s1),m)

NameError: name 'beta' is not defined